In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np

jobName = '201216_Train_split1'

# 단계6. 임베딩2 :  원핫 -> 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 임베딩_경과기간x현재나이 -> 스케일러
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetAttainedAge(x)
x = LibData.GetIrReduction(x)
x = LibEE.GetXIntersected(x, ['ElapsedMth','AttainedAge'], 'Inter_EM_AA')
x, emb = LibEE.GetXEmbApplied(3, x, y, ['ProdCode', 'Inter_EM_AA'])
scaler, x = LibUtils.ScaleStandard(x)

colsSelected = ['ElapsedMth', 'Sex', 'Prem', 'PremYr', 'AccumPrem', 'ContAmt', 'StartAge', 'FundAllo03', 'IR06', 'IR09', 'IR12', 'IR14', 'Moneyness', 'AttainedAge', 'ee_ProdCode_00', 'ee_ProdCode_01', 'ee_ProdCode_02']
x = x[colsSelected]


     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 1.3MB 4.2MB/s 


In [2]:
from sklearn.preprocessing import PolynomialFeatures

x = pd.DataFrame(PolynomialFeatures(2).fit_transform(x))

In [5]:
from sklearn.linear_model import Ridge, Lasso

def ObjFunc_Ridge(alpha):

    model = Ridge(alpha=alpha)    
    scores = LibML.GetScoresCV(x,y,model,5)
    return -scores[0]

def ObjFunc_Lasso(alpha):

    model = Lasso(alpha=alpha)    
    scores = LibML.GetScoresCV(x,y,model,5)
    return -scores[0]



In [9]:
# bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

pbounds = { 'alpha': (0, 1) }

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
bo=BayesianOptimization(f=ObjFunc_Ridge, pbounds=pbounds, verbose=2, random_state=1)

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)

bo.maximize(init_points=5, n_iter=50, acq='ei', xi=0.03)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(bo.max)

|   iter    |  target   |   alpha   |
-------------------------------------
start learning 1 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 2 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 3 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 4 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 5 2021-01-16 16:34... end learning 2021-01-16 16:34
|  1        | -328.7    |  0.417    |
start learning 1 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 2 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 3 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 4 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 5 2021-01-16 16:34... end learning 2021-01-16 16:34
|  2        | -328.8    |  0.7203   |
start learning 1 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 2 2021-01-16 16:34... end learning 2021-01-16 16:34
start learning 3 2021-01-16 16:34... end lea

KeyboardInterrupt: ignored

In [10]:
# bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

pbounds = { 'alpha': (0, 5) }

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
bo=BayesianOptimization(f=ObjFunc_Lasso, pbounds=pbounds, verbose=2, random_state=1)

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)

bo.maximize(init_points=5, n_iter=50, acq='ei', xi=0.03)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(bo.max)

|   iter    |  target   |   alpha   |
-------------------------------------
start learning 1 2021-01-16 16:38... end learning 2021-01-16 16:39
start learning 2 2021-01-16 16:39... end learning 2021-01-16 16:39
start learning 3 2021-01-16 16:39... end learning 2021-01-16 16:40
start learning 4 2021-01-16 16:40... end learning 2021-01-16 16:40
start learning 5 2021-01-16 16:40... end learning 2021-01-16 16:40
|  1        | -335.5    |  2.085    |
start learning 1 2021-01-16 16:40... end learning 2021-01-16 16:41
start learning 2 2021-01-16 16:41... end learning 2021-01-16 16:41
start learning 3 2021-01-16 16:41... end learning 2021-01-16 16:42
start learning 4 2021-01-16 16:42... end learning 2021-01-16 16:42
start learning 5 2021-01-16 16:42... end learning 2021-01-16 16:42
|  2        | -341.9    |  3.602    |
start learning 1 2021-01-16 16:42... 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 413415298.1212486, tolerance: 2150423.4694871507
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13584775085.418814, tolerance: 11448413.374285994
  positive)


end learning 2021-01-16 16:44
start learning 2 2021-01-16 16:44... 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 409889012.8206551, tolerance: 2158824.5760924327
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13297186564.24062, tolerance: 11326127.638646042
  positive)


end learning 2021-01-16 16:45
start learning 3 2021-01-16 16:45... 

...

end learning 2021-01-16 21:15
start learning 4 2021-01-16 21:15... 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7306584.40709722, tolerance: 2156471.613283809
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 647458515.7068748, tolerance: 11756342.9200159
  positive)


end learning 2021-01-16 21:16
start learning 5 2021-01-16 21:16... 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10402078.047513247, tolerance: 2159365.5218561306
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 924872451.5028915, tolerance: 11530368.28085911
  positive)


end learning 2021-01-16 21:17
|  55       | -329.4    |  0.2004   |
{'target': -329.02, 'params': {'alpha': 0.18445033718722081}}
